<a href="https://colab.research.google.com/github/Stanfording/small_GAN_testing/blob/main/64_cele_small_try.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# Goal:   Just get a small GAN running



Get the preprocessed celebHD data from google drive

In [ ]:
# #Download the dataset
# !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1KqBRLsB0CJuQGycvaPINwaPgcGDUsAxN' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1KqBRLsB0CJuQGycvaPINwaPgcGDUsAxN" -O "proCeleba.zip" && rm -rf /tmp/cookies.txt

# #unzip the dataset
# !unzip "/content/proCeleba.zip"

# #remove unnecessary files
# !rm -rf /content/__MACOSX

Import libraries

In [ ]:
from tqdm import tqdm

from torch.autograd import Variable, grad
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import tensorflow as tf
import os
import torch as t
import torch.nn as nn
from torchvision import datasets, transforms, utils
from torch.utils.data import Dataset, DataLoader
from skimage import io

Set initial hyperparameters

In [ ]:

img_fold_dir_64_reso = "/content/proCeleba/64"

batch_size = 64

resolution = 64

iteration = 10

device = t.device('cuda' if t.cuda.is_available() else 'cpu')

log_folder = "log"
!mkdir "log"

Show me which gpu I am using.

In [ ]:
!nvidia-smi -L

Pre loading the data

In [ ]:
# Define a data class for load unclassfied data.
class Get_No_Classes_Img_Dataset(Dataset):
    
    def __init__(self, folder_dir, transform = None):
        self.folder_dir = os.path.join(folder_dir)
        self.transform = transform
        self.image_list = os.listdir(self.folder_dir)
        
    def __len__(self):
        return len(os.listdir(self.folder_dir))
    
    def __getitem__(self, index):
        
        image_name = self.image_list[index]
        
        image_dir = os.path.join(self.folder_dir, image_name)
        
        image = io.imread(image_dir)
        
        if (self.transform != None):
            image = self.transform(image)
        
        return image
    

transform = transforms.ToTensor() #From Batch * Highth * Width * Channel to Batch * Channel * Highth * Width
                                  #Which is what pytorch CNN can work with.

datasets = Get_No_Classes_Img_Dataset(img_fold_dir_64_reso, transform = transform) 
                                                            # datasets[0].shape = (16,16,3)
                                                            # len(datasets) = 28000
total_data_len = len(datasets)
                                
# Viewing one data sample function:
def showOneImge(img, name = "", blur = False):
    
    img = img.squeeze()
    
    img = transforms.ToPILImage()(img)
    
    if blur:
        img = transforms.GaussianBlur(31, (100, 100))(img)
    plt.figure(figsize = (10,10), dpi = 10)
    plt.imshow(img)
    plt.axis('off')
    plt.savefig(f"/content/log{name}.png", bbox_inches='tight', pad_inches = 0)
    return


                                                            
loader = DataLoader(datasets, batch_size = batch_size)

loader = iter(loader)






```
print(next(loader).shape) 
```
will output


```
torch.Size([batch_size, 3, 64, 64])
```

So data loading is ready.

What's left is keep using 

```
next(loader)
```
to access each batch of data


-------------------------------------------------

Now Designing the simple GAN network

In [ ]:
"""
Define the generator
"""

class G(nn.Module):
    
    def __init__(self):
        super().__init__()

        self.laten = nn.Sequential(
            nn.Linear(5, 10),
            nn.Linear(10, 128),
            nn.Linear(128, 512),
            nn.Linear(512, 1024),
            nn.Linear(1024, 4096),
            nn.Linear(4096, 12288))
        self.net1 = nn.Conv2d(3, 16, (3, 3), padding = "same")
        self.net2 = nn.Conv2d(16, 64, (3, 3), padding = "same")
        self.net3 = nn.Conv2d(64, 64, (3, 3), padding = "same")
        self.net4 = nn.Conv2d(64, 16, (3, 3), padding = "same")
        self.net5 = nn.Conv2d(16, 3, (3, 3), padding = "same")
        
        
    def forward(self, theInput, batch_size):
        
        x = self.laten(theInput)
        
        x = t.reshape(x, (batch_size, 3, 64, 64))
         
        x = self.net1(x)
        x = self.net2(x)
        x = self.net3(x)
        x = self.net4(x)
        x = self.net5(x)
            
            
        return x
    
class D(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.netWork = nn.Sequential(
            nn.Conv2d(3, 16, (3, 3), padding = "same"),
            nn.Conv2d(16, 64, (3, 3), padding = "same"),
            nn.MaxPool2d((3,3)),
            nn.Conv2d(64, 64, (3, 3), padding = "same"),
            nn.MaxPool2d((3,3)),
            nn.Conv2d(64, 16, (3, 3), padding = "same"),
            nn.MaxPool2d((3,3)),
            nn.Flatten(),
            nn.Linear(64, 3),
            nn.LeakyReLU(),
            nn.Linear(3, 1),
            nn.Sigmoid())
        
    def forward(self, theInput):
        
        return self.netWork(theInput)

Testing the network

In [ ]:
"""

Testing the net work:

"""

# oneImg = datasets[0]

# showOneImge(oneImg)

# oneImg = oneImg.expand(1,3,64,64)

# print(oneImg.shape)

# #img into G to test shape

# input_noise_example = t.randn((batch_size, 1, 1, 5))

# a = G()(input_noise_example, batch_size)

# showOneImge(a[0])

# b = D()(next(loader))

# print(b.shape)

Gradient Penalty from wGAN, will test later.

In [ ]:
def compute_gradient_penalty(D, real_samples, fake_samples):
    """Calculates the gradient penalty loss for WGAN GP"""
    # Random weight term for interpolation between real and fake samples
    alpha = t.randn((batch_size, 1, 1, 1))
    # Get random interpolation between real and fake samples
    interpolates = (alpha * real_samples + ((1 - alpha) * fake_samples)).requires_grad_(True)
    
    
    d_interpolates = D()(interpolates)
    
    
    grad_x_hat = grad(
            outputs=d_interpolates.sum(), inputs=interpolates, create_graph=True)[0]
    grad_penalty = ((grad_x_hat.view(grad_x_hat.size(0), -1)
                      .norm(2, dim=1) - 1)**2).mean()
    grad_penalty = 10 * grad_penalty
    
    return grad_penalty

Initializing the generator, discriminator, optimizer, labels, and loss.

In [24]:
generator = G().to(device)
discriminator = D().to(device)


G_optimizer = t.optim.Adam(generator.parameters(), lr = 0.001, betas=(0.0, 0.99))
D_optimizer = t.optim.Adam(discriminator.parameters(), lr = 0.001, betas=(0.0, 0.99))

label_real = t.ones((batch_size, 1)).to(device)
label_fake = t.zeros((batch_size, 1)).to(device)

bceLoss = nn.BCELoss()

Start training

In [ ]:
for i in range(iteration):

    p = tqdm(range(total_data_len // batch_size + 1)) # This is a progress bar run on each epoch
    
    for j in p:
        
    
        batchNum = str(i)
        
        #Training the D
        
        real = next(loader).to(device)
        
        input_noise = t.randn((batch_size, 1, 1, 5)).to(device)
        fake = generator(input_noise, batch_size)
        
        real_score = discriminator(real)

        if (label_real.shape != real_score.shape):
          label_real = t.ones((real_score.shape)).to(device)
          


        d_real_loss = bceLoss(real_score, label_real)
        fake_score = discriminator(fake)

        if fake_score.shape != label_fake.shape:
          label_fake = t.zeros((real_score.shape)).to(device)

        d_fake_loss = bceLoss(fake_score, label_fake)
        
        total_loss = d_real_loss + d_fake_loss
        
        
        total_loss.backward()
        D_optimizer.step()
        
        discriminator.zero_grad()
        generator.zero_grad()
        
        #Training the G
        
        
        input_noise = t.randn((batch_size, 1, 1, 5)).to(device)
        fake = generator(input_noise, batch_size)
        fake_score = discriminator(fake)
        g_fake_loss = bceLoss(fake_score, label_fake)
        
        g_fake_loss.backward()
        G_optimizer.step()
        
        discriminator.zero_grad()
        generator.zero_grad()
        
        mse = "Epoch: " + batchNum
        
        p.set_description(mse)
            
        p.set_postfix(G_loss = g_fake_loss.item(), D_total_loss = total_loss.item(), real_score = real_score.mean().item(), fake_score = fake_score.mean().item())
        
        
    
    if i % 5 == 0:
        input_noise = t.randn((batch_size, 1, 1, 5)).to(device)
        fake = generator(input_noise, batch_size)
        
        
        showOneImge(fake[0], name = "generator_epoch_" + str(i+1), blur = False)
        print("epoch = ", i + 1)
        
        t.save(generator.state_dict(), f'{log_folder}/checkpoint/{str(i + 1).zfill(6)}_g.model')
        t.save(discriminator.state_dict(), f'{log_folder}/checkpoint/{str(i + 1).zfill(6)}_d.model')


Epoch: 0:  13%|█▎        | 57/438 [00:23<02:32,  2.49it/s, D_total_loss=0.00016, G_loss=0, fake_score=2.77e-12, real_score=1]